## 3D Implied Volatility Surface
Load `options_data.csv` and visualize implied volatility as a 3D scatter plot.

In [30]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

pd.options.display.float_format = lambda x: f'{x:.4f}'


In [31]:
CSV_PATH = 'options_data.csv'

df_raw = pd.read_csv(CSV_PATH)
required_cols = {'S0', 'K', 'T', 'C_mkt', 'iv'}
missing = required_cols - set(df_raw.columns)
if missing:
    raise ValueError(f'Input CSV is missing columns: {missing}')

spot = float(df_raw['S0'].median())
lower_bound = np.ceil((spot - 100.0) / 10.0) * 10.0
upper_bound = np.ceil((spot + 100.0) / 10.0) * 10.0
mask = (df_raw['K'] >= lower_bound) & (df_raw['K'] <= upper_bound)
df = df_raw.loc[mask].sort_values(['T', 'K']).reset_index(drop=True)
if df.empty:
    raise ValueError('No strikes within the specified window.')

print(
    f"Spot ~ {spot:.2f}. Keeping strikes in [{lower_bound:.2f}, {upper_bound:.2f}] => {len(df)} rows."
)


Spot ~ 671.93. Keeping strikes in [580.00, 780.00] => 489 rows.


In [32]:
k_values = np.sort(df['K'].unique())
t_values = np.sort(df['T'].unique())

surface = df.pivot_table(index='T', columns='K', values='iv', aggfunc='mean')
surface = surface.reindex(index=t_values, columns=k_values)
surface = surface.interpolate(axis=1, limit_direction='both').interpolate(axis=1, limit_direction='both')
surface = surface.loc[surface.index >= 0.1]

IV_surface = surface.to_numpy(dtype=float)

if np.isnan(IV_surface).any():
    IV_surface = np.where(np.isnan(IV_surface), np.nanmean(IV_surface), IV_surface)


In [33]:
k_values = surface.columns.to_numpy(dtype=float)
t_values = surface.index.to_numpy(dtype=float)
KK, TT = np.meshgrid(k_values, t_values)

fig = go.Figure(
    data=[
        go.Surface(
            x=KK,
            y=TT,
            z=IV_surface,
            colorscale='Viridis',
            colorbar=dict(title='IV'),
            showscale=True,
        )
    ]
) 

fig.update_layout(
    title='Interpolated Implied Volatility Surface (S0 ± 100 strikes)',
    scene=dict(
        xaxis_title='Strike K',
        yaxis_title='Time to Maturity T (years)',
        zaxis_title='Implied Volatility',
    ),
    width=900,
    height=600,
)
fig.show()

